In [2]:
import pandas as pd
import os

# ⌨️ Entrées utilisateur
repertoire_entree = input("📁 Entrez le répertoire d'entrée : ").strip()
repertoire_sortie = input("📁 Entrez le répertoire de sortie : ").strip()
finess = input("🔢 Entrez le numéro FINESS : ").strip()
liste_annees = [2024, 2023, 2022]

# 🧾 Pour suivi
fichiers_traites = []
fichiers_non_trouves = []

for annee in liste_annees:
    try:
        # 📥 Chemins d'entrée
        fichier_rpsa = os.path.join(repertoire_sortie, f"RPSA.Filtre.{finess}.{annee}.txt")
        fichier_tra_raa = os.path.join(repertoire_sortie, f"TRA.RAA.{finess}.{annee}.crypte.txt")
        fichier_r3a = os.path.join(repertoire_entree, f"{finess}.{annee}.12.r3a")

        # 📤 Chemin de sortie
        sortie_r3a_filtre = os.path.join(repertoire_sortie, f"R3A.Filtre.{finess}.{annee}.txt")

        # 🔄 Chargement
        RPSA = pd.read_csv(fichier_rpsa, names=["colRPSA"])
        TRA_RAA = pd.read_csv(fichier_tra_raa, names=["colTRA"])
        R3A = pd.read_csv(fichier_r3a, names=["colR3A"])

        # 🔍 Extraction & filtrage
        RPSA["IPP"] = RPSA["colRPSA"].str[-24:]
        TRA_RAA["IPP"] = TRA_RAA["colTRA"].str[:24]
        TRA_RAA["num_ordre"] = TRA_RAA["colTRA"].str[43:51]
        R3A["num_ordre"] = R3A["colR3A"].str[106 :114]

        liste_uniques_ipp = RPSA["IPP"].unique().tolist()
        TRA_RAA_filtré = TRA_RAA[TRA_RAA["IPP"].isin(liste_uniques_ipp)]
        liste_num_ordre_uniques = TRA_RAA_filtré["num_ordre"].unique().tolist()

        R3A_filtré = R3A[R3A["num_ordre"].isin(liste_num_ordre_uniques)]

        # 🧩 Ajouter IPP depuis TRA_RAA_filtré via correspondance num_ordre
        mapping_num_ordre_ipp = TRA_RAA_filtré.drop_duplicates("num_ordre")[["num_ordre", "IPP"]]
        R3A_filtré = R3A_filtré.merge(mapping_num_ordre_ipp, on="num_ordre", how="left")

        # 💾 Sauvegarde du seul fichier filtré nécessaire
        R3A_filtré[["colR3A", "IPP"]].to_csv(sortie_r3a_filtre, index=False, header=None)

        fichiers_traites.append(f"✅ {annee}")
    except FileNotFoundError as e:
        fichiers_non_trouves.append(f"❌ {annee} - {e.filename} non trouvé")
    except Exception as e:
        fichiers_non_trouves.append(f"❌ {annee} - Erreur : {str(e)}")

# 📋 Résumé
print("\n📦 Fichiers traités avec succès :")
for f in fichiers_traites:
    print(f)

print("\n🚫 Fichiers non trouvés ou erreurs :")
for f in fichiers_non_trouves:
    print(f)

📁 Entrez le répertoire d'entrée :  C:\Users\Youba\Documents
📁 Entrez le répertoire de sortie :  C:\Users\Youba\Documents
🔢 Entrez le numéro FINESS :  20001



📦 Fichiers traités avec succès :

🚫 Fichiers non trouvés ou erreurs :
❌ 2024 - C:\Users\Youba\Documents\20001.2024.12.r3a non trouvé
❌ 2023 - C:\Users\Youba\Documents\RPSA.Filtre.2023.txt non trouvé
❌ 2022 - C:\Users\Youba\Documents\RPSA.Filtre.2022.txt non trouvé
❌ 2021 - C:\Users\Youba\Documents\RPSA.Filtre.2021.txt non trouvé


***Préparé par : Lekehal Hammada***

***Équipe : I4S***

***30/04/2025***